In [2]:
#import modules
import numpy as np
import pandas as pd
import re
import itertools
import glob

#Snapgene module
from snapgene_reader import snapgene_file_to_dict, snapgene_file_to_seqrecord

#Functions
def band_filter(distance):
    accuracy_of_fragment_analzyer = 0.1
    flag = 1

    if max(distance) > 9500:
        flag = flag*0
    else:
        flag = flag*1
    
    if min(distance) < 100:
        flag = flag*0
    else:
        flag = flag*1
    
    for i in range(len(distance)-1):
        band_size_diff = distance[i+1] - distance[i]
        if band_size_diff > accuracy_of_fragment_analzyer*distance[i]:
            flag = flag*1
        else:
            flag = flag*0

    if flag == 1:
        return True
    else: 
        return False

def list_sort(curr_list): 
    curr_list.sort(key = lambda x: x[0]) 
    return pd.DataFrame(curr_list,columns = ['fragment_number','dna_sequence'])

def revcom(seq):
    answer=''
    for i in range(0,len(seq)):       
        if seq[i]== 'A' or seq[i]== 'a':
            answer ='T'+answer
        elif seq[i]=='C' or seq[i]=='c':
            answer ='G'+answer
        elif seq[i]=='G' or seq[i]=='g':
            answer ='C'+answer
        elif seq[i]=='T' or seq[i]=='t':
            answer ='A'+answer
    return answer

#Restriction Enzyme for Verification
df = pd.read_excel("common_re_list.xlsx")
re_site = df.to_numpy()
count = 0
for re_i in range(1,np.shape(re_site)[0]):
    curr_re_seq = re_site[re_i,0]
    if curr_re_seq == revcom(curr_re_seq):
        count = count + 1
        
if count == len(re_site)-1:
    print("All restriction enzymes are palindromic") 
    
final_re_count = 0
common_final_re_df_index = ['RE Combinations']
for files in glob.glob("*.dna"):
    snap_file = files  
    
    #reading snap_gene_file
    dictionary = snapgene_file_to_dict(snap_file)
    seqrecord = snapgene_file_to_seqrecord(snap_file)
    filename = snap_file.split('.')[0] + '.xlsx'

    count = 0
    data_stat = []
    for i in range(len(dictionary['features'])):
        if 'fragment' in dictionary['features'][i]['name']:
            count = count + 1
            frag_start, frag_end = next(iter(dictionary['features'][i]['segments'][0].items()))[1].split('-')
            f_start = int(frag_start)-1
            f_end = int(frag_end)
            if f_start < f_end:
                curr_seq = dictionary['seq'][f_start:f_end]
            else:
                curr_seq = dictionary['seq'][f_start:len(dictionary['seq'])] + dictionary['seq'][0:f_end]
            data_stat.append([int(dictionary['features'][i]['name'].replace('fragment','')),curr_seq])
            
        if 'Fragment' in dictionary['features'][i]['name']:
            count = count + 1
            frag_start, frag_end = next(iter(dictionary['features'][i]['segments'][0].items()))[1].split('-')
            f_start = int(frag_start)-1
            f_end = int(frag_end)
            if f_start < f_end:
                curr_seq = dictionary['seq'][f_start:f_end]
            else:
                curr_seq = dictionary['seq'][f_start:len(dictionary['seq'])] + dictionary['seq'][0:f_end]
            data_stat.append([int(dictionary['features'][i]['name'].replace('Fragment','')),curr_seq])

    df = list_sort(data_stat)
    insert_order_specified = np.array(df.fragment_number)
    sequences_specified = np.array(df.dna_sequence)

    order = insert_order_specified.argsort()
    insert_order = insert_order_specified[order]
    sequences_case = sequences_specified[order]

    pre_sequences = []
    for i in range(len(sequences_case)):
        pre_sequences.append(sequences_case[i].upper())
    
    del_index = []
    merge_flag = 0
    merging_tm_template = []
    for i in range(len(pre_sequences)):
        if len(pre_sequences[i])<80:
            del_index.append(i)
            print('fragment less than 80 bp')
            merge_flag = 1
            if len(pre_sequences[i])%2 == 0:
                first_half = pre_sequences[i][0:len(pre_sequences[i])//2]
                second_half = pre_sequences[i][len(pre_sequences[i])//2:]
            else:
                first_half = pre_sequences[i][0:len(pre_sequences[i])//2]
                second_half = pre_sequences[i][len(pre_sequences[i])//2:]

            if i==0:
                merging_tm_template.append([len(pre_sequences)-1,i+1])     

                pre_sequences[len(pre_sequences)-1] = pre_sequences[len(pre_sequences)-1] + first_half
                pre_sequences[i+1] = second_half + pre_sequences[i+1]

            elif i==len(pre_sequences)-1: 
                merging_tm_template.append([i-1,0])   

                pre_sequences[i-1] = pre_sequences[i-1] + first_half
                pre_sequences[0] =  second_half + pre_sequences[0]
            else:
                merging_tm_template.append([i-1,i+1])   

                pre_sequences[i-1] = pre_sequences[i-1] + first_half
                pre_sequences[i+1] = second_half + pre_sequences[i+1]

    sequences = []
    if len(del_index) > 0:
        for i in range(len(pre_sequences)):
            if len(pre_sequences[i]) >= 80:
                sequences.append(pre_sequences[i])
    else:
        sequences = pre_sequences

    #Restoring presequence (by removing if there was any addition)
    if merge_flag == 1:
        pre_sequences = []
        for i in range(len(sequences_case)):
            pre_sequences.append(sequences_case[i].upper())

    ##Complete Plasmid Sequence
    for p_i in range(len(sequences)):
        if p_i == 0:
            plasmid_seq = sequences[p_i]
        else:
            plasmid_seq = plasmid_seq + sequences[p_i]

    re_plasmid_seq = plasmid_seq + sequences[0][0:7]
    str_plasmid_seq = plasmid_seq + sequences[0][0:21] #to ensure we analyze circular plasmid 
    op_str_plasmid_seq = revcom(str_plasmid_seq)
    
    pre_verify_re = np.array(['enzyme name','recognition sequence','cut_index','frequency'])
    for re_i in range(np.shape(re_site)[0]):
        curr_re_seq = re_site[re_i,0]
        re_frequency = len([m.start() for m in re.finditer(curr_re_seq, re_plasmid_seq)])
        pre_verify_re = np.vstack((pre_verify_re,[re_site[re_i,3],curr_re_seq,re_site[re_i,2],re_frequency]))    

    verify_re = pre_verify_re[0,:]
    for i in range(1,np.shape(pre_verify_re)[0]):
        if pre_verify_re[i,3] != '0':
            verify_re = np.vstack((verify_re,pre_verify_re[i,:]))
    
    length_comb = 4 #int(input("Enter number of restriction enzymes you want to use: [Higher number recommended for smaller plasmid]" + "Enter a number less than " + str(np.shape(verify_re)[0]) +":"))
    re_final = ['RE combination','Band Size']   
    comb_to_explore = np.arange(length_comb)
    for p in range(len(comb_to_explore)):
        re_options = []
        re_found = False
        for re_opt_i in itertools.combinations(verify_re[1:,0],p+1):
            re_options.append(np.asarray(re_opt_i))

        re_gel_comb = []
        for l in range(p+1):
            re_gel_comb.append('RE'+ str(l+1))

        re_gel_comb.append('band sizes')
        for i in range(np.shape(re_options)[0]):
            re_names = re_options[i]
            locations = []
            for j in range(len(re_names)):
                index = np.where(verify_re[:,0] == re_options[i][j])[0][0]
                matches = re.finditer(verify_re[index,1], re_plasmid_seq)
                locations.append([match.start() for match in matches])

            re_locations = locations[0]
            if len(locations) > 1:
                for m in range(len(locations)-1):
                    re_locations = np.concatenate([re_locations,locations[m+1]])

            re_locations = np.sort(re_locations)
            band_size = []
            for j in range(len(re_locations)):
                if j == len(re_locations) - 1:
                    band_size.append(len(plasmid_seq)-re_locations[j]+re_locations[0])
                else:
                    band_size.append(re_locations[j+1]-re_locations[j])

            if band_filter(np.sort(band_size)):
                re_found = True
                if len(band_size) > 1:
                    band_size_trans = str(';'.join(str(v) for v in list(np.sort(band_size))))
                    re_gel_comb = np.vstack((re_gel_comb,np.concatenate([re_names,[band_size_trans]])))

        if np.array(re_gel_comb).ndim > 1:
            for m in range(np.shape(re_gel_comb)[0]-1):
                curr_re_comb = ''
                for k in range(p+1):
                    curr_re_comb = curr_re_comb + re_gel_comb[m+1,k] + ', '

                re_final = np.vstack((re_final,[curr_re_comb[:-2],re_gel_comb[m+1,np.shape(re_gel_comb)[1]-1]]))

        if p == len(comb_to_explore)-1:
            final_re_df = pd.DataFrame(re_final)
            if final_re_count == 0:
                common_final_re_df = final_re_df[1:]
            else:
                common_final_re_df = pd.merge(common_final_re_df, final_re_df[1:], how='inner', on = 0)

            final_re_count = 1
            common_final_re_df_index.append(snap_file.split('.')[0])

common_final_re_df.columns = common_final_re_df_index
common_final_re_df.to_csv('Combined_RE_for_verification_only'+'.csv', index=False)

All restriction enzymes are palindromic
fragment less than 80 bp
fragment less than 80 bp
